<b><font size="6">Machine Learning</font><a class="anchor"><a id='toc'></a></b><br>

# <font color='54C13C'>3rd Individual Assignment</font> <a class="anchor" id="first-bullet"></a>

<div class="alert alert-block alert-info">  
    
### Student Name:
Sofia Alexandra Grifo Cunha
### Student Number:
M20190795 
</div>

## <font color="54C13C"> Company Dataset </font> <a>

###  Data Description

 `Employee_ID`: The unique identifier of the employee\
 `Gender`: The gender of the employee\
 `Age`: The age (in years) of the employee\
 `Days_off`:  Number of days the employee took off over the last year\
 `Rotations`: Number of team changes the employee made\
 `Satis_leader`: Level of satisfaction with the leadership\
 `Satis_team`: Level of satisfaction with the team\
 `Emails`: Average number of received emails in a day\
 `Tenure`: Number years the employee has been working for the company\
 `Bonus`: Bonus received as percentage of salary\
 `Distance`: Distance from home to the workplace\
`Kids`: Number of kids\
 `Overtime`: Average number of overtime hours a month\
 `Marital_status`: Marital status of the employee\
 `Department`: Department of the employee\
 `Churn_risk`: Churn risk level of the employee\

### <font color="54C13C"> Imports </font> <a>

In [1]:
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings('ignore')
import time
from sklearn.model_selection import KFold
import numpy as np
from sklearn import tree
import csv
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import (train_test_split,
                                     KFold,
                                     GridSearchCV)
from sklearn.preprocessing import (Normalizer,
                                   MinMaxScaler,
                                   LabelEncoder,
                                   RobustScaler,
                                   StandardScaler)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import f1_score


### <font color="54C13C"> Functions </font> <a>

In [2]:
def avg_score(model):
    kf = KFold(n_splits=10)
    score_train = []
    score_test = []
    timer = []
    for train_index, test_index in kf.split(X_company):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        begin = time.perf_counter()
        model.fit(X_train, y_train)
        end = time.perf_counter()
        value_train = model.score(X_train, y_train)
        value_test = model.score(X_test,y_test)
        score_train.append(value_train)
        score_test.append(value_test)
        timer.append(end-begin)

    avg_time = round(np.mean(timer),3)
    avg_train = round(np.mean(score_train),3)
    avg_test = round(np.mean(score_test),3)
    std_time = round(np.std(timer),2)
    std_train = round(np.std(score_train),2)
    std_test = round(np.std(score_test),2)
    
    return str(avg_time) + '+/-' + str(std_time), str(avg_train) + '+/-' + str(std_train),\
str(avg_test) + '+/-' + str(std_test)

In [3]:
from sklearn.tree import export_graphviz
import graphviz
import pydotplus

def plot_tree(model):
    dot_data = export_graphviz(model,
                               feature_names=X_train.columns,  
                               class_names=["Low", "Medium", "High"],
                               filled=True)
    pydot_graph = pydotplus.graph_from_dot_data(dot_data)
    pydot_graph.set_size('"20,20"')
    return graphviz.Source(pydot_graph.to_string())

In [4]:
def Age(data): 
    data['Age'] = pd.cut(data['Age'], bins=[0, 19, 29, 39, 49, 59, 99], labels=[0, 1, 2, 3, 4, 5])
    data['Age'].value_counts(normalize = True)

In [5]:
def show_results(df, *args):
    """
    Receive an empty dataframe and the different models and call the function avg_score
    """
    count = 0
    # for each model passed as argument
    for arg in args:
        # obtain the results provided by avg_score
        time, avg_train, avg_test = avg_score(arg)
        # store the results in the right row
        df.iloc[count] = time, avg_train, avg_test
        count+=1
    return df

## <font color="54C13C"> Train Set </font> <a>

### <font color="54C13C">  Data Understanding </font> <a>

I use the dataset train to build my model. Then I create an objectsnamed __company__ that contains the dataset __train.csv__.

In [6]:
company = pd.read_csv('train.csv')

### <font color="54C13C"> Transformations </font> <a>

Due to the fact that we cant scalle categorical variables, I decided to replace the categories, so this variable transforms into a numerical one.

In [7]:
company['Churn_risk'] = company['Churn_risk'].replace('low', 0)
company['Churn_risk'] = company['Churn_risk'].replace('medium', 1)
company['Churn_risk'] = company['Churn_risk'].replace('high', 2)

**Missing values**:

In [8]:
company.isnull().sum()

Employee_ID       0
Gender            0
Age               9
Days_off          8
Rotations         2
Satis_leader      2
Satis_team        1
Emails            0
Tenure            0
Bonus             0
Distance          9
Kids              4
Overtime          0
Marital_status    0
Department        0
Churn_risk        0
dtype: int64

After analysing the missing values that exist in the dataset, I replace them by 0 and by the mean to slove it.

In [9]:
company.loc[company.Days_off.isnull(), 'Days_off'] = 0
company.loc[company.Rotations.isnull(), 'Rotations'] =0
company.loc[company.Kids.isnull(), 'Kids'] = 0
company.loc[company.Distance.isnull(), 'Distance'] = company.Distance.mean()
company.loc[company.Age.isnull(), 'Age'] = company.Age.mean()
company.loc[company.Satis_leader.isnull(), 'Satis_leader'] = company.Satis_leader.mean()
company.loc[company.Satis_team.isnull(), 'Satis_team'] = company.Satis_team.mean()

`Gender`, `Department`and `Marital_status`are categorical variables, so in a way to solve this problem for the futher analysis, my solution was to transform into dummys.

In [10]:
categoricals = ['Gender','Department', 'Marital_status']
new_categoricals = [col for col in categoricals if col in company.columns]

company = pd.get_dummies(data=company, columns=new_categoricals)

In [11]:
company['Age'] = company['Age'].astype('int64')

**Outliers**:

In [12]:
out = company.loc[company['Days_off']<0]
out = out.append(company.loc[company['Days_off']>24])
out = out.append(company.loc[company['Age']>=68])
out = out.append(company.loc[company['Satis_leader']>5])
out = out.append(company.loc[company['Satis_team']>8])
out = out.append(company.loc[company['Emails']>120])
out = out.append(company.loc[company['Tenure']>35])
out = out.append(company.loc[company['Bonus']>2])
out = out.append(company.loc[company['Distance']>4])
out = out.append(company.loc[company['Overtime']>14])
company = company.loc[~company['Employee_ID'].isin(out['Employee_ID'])]

In [13]:
company.set_index('Employee_ID', inplace = True)

I create an object named __X_company__ that will contain the independent variables and another object named __y_company__ that will contains the independent varaible. And it was used the train_test_slipt method to slipt the data into the train and test sets. The divison was with 80% of the data to build the model, and 20% to evaluate the performance of the final model.

In [14]:
X_company= company.drop(['Churn_risk'], axis = 1)
y_company = company['Churn_risk']

X_train_company, X_test_company, y_train_company, y_test_company= train_test_split(X_company, y_company,
                                                                                   test_size = 0.2, random_state = 15,
                                                                                   stratify = y_company)

## <font color="54C13C"> Decision Tree Model </font> <a>

**Classifier**: I create an instace of DecisionTreeClassifie named DTmodel. And after I use the .fit()method of model to fit the model to the array of points X_train_company and y_train_company. 

In [15]:
DTmodel= DecisionTreeClassifier(ccp_alpha=0.0007622811970638056, class_weight=None,
                       criterion='gini', max_depth=None, max_features=None,
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       presort='deprecated', random_state=0, splitter='best').fit(X_train_company, y_train_company).fit(X_train_company, y_train_company)

In [16]:
DTmodel.score(X_train_company, y_train_company)

0.8181159420289855

In [17]:
DTmodel.score(X_test_company, y_test_company)

0.7393822393822393

In [18]:
predictDT= DTmodel.predict(X_test_company)
print (classification_report (y_test_company, predictDT))

              precision    recall  f1-score   support

           0       0.68      0.67      0.67       379
           1       0.77      0.81      0.79       625
           2       0.67      0.12      0.21        32

    accuracy                           0.74      1036
   macro avg       0.71      0.54      0.56      1036
weighted avg       0.74      0.74      0.73      1036



In [19]:
print(f1_score(y_test_company, predictDT, average='weighted'))

0.7316190305265563


__Final Result:__ To evaluate my model I had in account the value from the score, where I got 0.74, and the F1_score where the result was 0.73

## <font color="54C13C"> Test Set </font> <a>

I use the test to see how well my model performs on unseen data. Then I create one objects named __testcompany__ which contains the dataset __test.csv__.

In [20]:
testcompany = pd.DataFrame(pd.read_csv("test.csv"))

### <font color="54C13C"> Transformations </font> <a>

In [21]:
tc = testcompany.copy()

In [22]:
tc.set_index('Employee_ID', inplace = True)

**Missing values**: After analysing the missing values that exist in the dataset, I replace them by 0 and bt the mean to slove this problem.

In [23]:
tc.loc[tc.Days_off.isnull(), 'Days_off'] = 0
tc.loc[tc.Satis_team.isnull(), 'Satis_team'] =tc.Satis_team.mean()
tc.loc[tc.Age.isnull(), 'Age'] = tc.Age.mean() 
tc.loc[tc.Rotations.isnull(), 'Rotations'] = 0

`Gender`, `Department`and `Marital_status`are categorical variables, so in a way to solve this problem for the futher analysis, my solution was to transform into dummys.

In [24]:
categoricals = ['Gender','Department', 'Marital_status']
new_categoricals = [col for col in categoricals if col in tc.columns]

tc = pd.get_dummies(data=tc, columns=new_categoricals)

### <font color="54C13C"> Test prediction </font> <a>

After having the train results of my model, the next step was to predict it in the test dataset, wich after the transformations is named __tc__. 

In [25]:
prev = DTmodel.predict(tc)
predict= pd.DataFrame()
predict['Employee_ID']= testcompany['Employee_ID'].copy()
predict['Churn_risk']= prev

In [26]:
predict['Churn_risk'] = predict['Churn_risk'].replace(0, 'low')
predict['Churn_risk'] = predict['Churn_risk'].replace(1, 'medium')
predict['Churn_risk'] = predict['Churn_risk'].replace(2, 'high')

In [27]:
predict

,Employee_ID,Churn_risk
0,1005201,low
1,1005202,medium
2,1005203,low
3,1005204,medium
4,1005205,medium
...,...,...
1295,1006496,medium
1296,1006497,medium
1297,1006498,low
1298,1006499,medium


### <font color="54C13C"> Export .csv file </font> <a>

In [28]:
predict.to_csv('M20190795_DTversion18.csv',index=False, header=True)